In [1]:
from bs4 import BeautifulSoup
from lxml import etree as ET
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
import requests
from bs4 import BeautifulSoup as soup
import polars as pl
#from arcgis import GIS
#import geopy
#from geopy.geocoders import Nominatim

#from arcgis.features import SpatialDataFrame
#from arcgis.features import GeoAccessor
#from arcgis.gis import GIS

In [2]:
dump = pl.read_parquet('data/online_diss_cleaned_03-2024.parquet') 
data = dump.to_pandas()
#data = pd.read_csv('data/check-data-csv.csv')
data

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008
0,1000000389,"Klöfkorn, Robert",Numerics for evolution equations : a general i...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,510,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
1,1000006395,"Speckmann, Bodo",Regulation der hepatischen Biosynthese von Sel...,ger,Düsseldorf,2009,570,570,DE-101,573.3845554,540,"Düsseldorf, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
2,100000645X,"Sauheitl, Leopold",Direct amino acid uptake by plants related to ...,eng,Bayreuth,2009,N/A,N/A,N/A,570,N/A,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
3,1000006948,"Sanchez, Martin",Charakterisierung des Replikationsenzyms ORF90...,ger,Bayreuth,2010,570,570,DE-101,572.86453629321,N/A,"Bayreuth, Univ., Diss., 2010",N/A,100204s2009 gw |||||om||| 00||||ger
4,1000007138,"Reimann, Bodo",Numerische Untersuchung der Stoßrohrströmung ...,ger,Duisburg,2009,620,620,DE-101,629.4152,620,"Duisburg, Essen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324777,999997297,"Moschkau, Peter",Defektabhängige Transporteigenschaften von Pr...,ger,Göttingen,2009,N/A,N/A,N/A,530,N/A,"Göttingen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
324778,999997505,"Scheler, Esther",Tailoring fluorene-based oligomers for micron ...,eng,Bayreuth,2009,540,540,DE-101,547.70455,VN 5927,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
324779,99999767X,"Körzdörfer, Thomas",Self-interaction and charge transfer in organi...,eng,Bayreuth,2009,530,530,DE-101,537.6223,UM 1200,"Bayreuth, Univ., Diss., 2009",Enth. 4 Sonderabdr. aus verschiedenen Zeitschr.,100204s2009 gw |||||om||| 00||||eng
324780,999999389,"Xystrakis, Fotios",The drought tolerance limit of European beec...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,630,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",Dateien in unterschiedlichen Formaten,100204s2009 gw |||||om||| 00||||eng


In [3]:
data.loc[data['Place'] == "N/A"]

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008


In [4]:
##Um Probleme mit Sonderzeichen beim Geocodieren zu vermeiden, normalisieren wir noch einmal das Dataframe mit Hilfe einer Funktion: 

def normalize(text): 
    new = unicodedata.normalize('NFC', text)
    return new 

data['Place'] = data['Place'].apply(normalize)
data['Place'] = data['Place'].str.strip('[]')
#df['Place'].replace("Karlsruher Inst. für Technologie", "Karlsruhe", inplace=True)
#df['Place'].replace("Hagen. Fernuniv.", "Hagen", inplace=True)

df = data[["ID", "Title", "Place"]]
df = df.drop_duplicates("Place")
#df.to_csv("places_to_reconcile.csv")
df

,ID,Title,Place
0,1000000389,Numerics for evolution equations : a general i...,Freiburg (Breisgau)
1,1000006395,Regulation der hepatischen Biosynthese von Sel...,Düsseldorf
2,100000645X,Direct amino acid uptake by plants related to ...,Bayreuth
4,1000007138,Numerische Untersuchung der Stoßrohrströmung ...,Duisburg
5,1000007820,Synthese funktionalisierter Trinukleotide und ...,Greifswald
...,...,...,...
320835,997168250,Quarkonium states in an anisotropic quark gluo...,Frankfurt (Main) Univ.
321156,997391057,Climatic and hydrographic variability in the l...,Bremenhaven
321178,997404221,Aufbau und Evaluation stationärer Verhaltenst...,Dortmund
324293,999726307,Eine systemorientierte Managementmethode fü...,Oberpframmern


In [5]:
df.loc[df['Place'] == "N/A"]

,ID,Title,Place


### Im nächsten Schritt wurde mit Hilfe von OpenRefine und der Lobid-Reconciliation-API der Normdatensatz in der GND für jedes Geografikum bestimmt und diesem zugeordnet. Im folgenden wird die um GND-ID erweiterte Tabelle geladen und auf die ursprüngliche Liste aller Publikationen gemappt. 

In [11]:
gnd_places = pd.read_excel("data/places-with-gnd.xlsx")
#gnd_places.loc[gnd_places['Place'] == "N/A"]
gnd_places

,Column,ID,Title,Place,Abgleich,GND,GND-ID
0,190.0,1000000389,Numerics for evolution equations : a general i...,Freiburg (Breisgau),Freiburg im Breisgau,Freiburg im Breisgau,4018272-1
1,191.0,1000006395,Regulation der hepatischen Biosynthese von Sel...,Düsseldorf,Düsseldorf,Düsseldorf,4013255-9
2,192.0,100000645X,Direct amino acid uptake by plants related to ...,Bayreuth,Bayreuth,Bayreuth,4005056-7
3,194.0,1000007138,Numerische Untersuchung der Stoßrohrströmung ...,Duisburg,Duisburg,Duisburg,4013284-5
4,195.0,1000007820,Synthese funktionalisierter Trinukleotide und ...,Greifswald,Greifswald,Greifswald,4021965-3
...,...,...,...,...,...,...,...
265,311511.0,999942581,Prozesseinflüsse auf die Kompatibilität von ...,Gauting,Gauting,Gauting,4092713-1
266,NaN,1300193115,A framework to foster traceability of E/E arti,Enschede,Enschede,Enschede,4014830-0
267,NaN,NaN,NaN,Montreal,Montréal,Montréal,4040188-1
268,NaN,NaN,NaN,Auerbach/Voigtland,Auerbach/Vogtl.,Auerbach/Vogtl.,4278974-6


In [12]:
new_places = df.merge(gnd_places, on='Place', indicator=True)
new_places.loc[new_places['_merge'] != "both"]

,ID_x,Title_x,Place,Column,ID_y,Title_y,Abgleich,GND,GND-ID,_merge


In [17]:
new_places.loc[new_places['Abgleich'] != new_places['GND']]

,ID_x,Title_x,Place,Column,ID_y,Title_y,Abgleich,GND,GND-ID,_merge
62,1000466175,Narrative Unterrichtsformen im Englischunterri...,Weingarten,1299.0,1000466175,Narrative Unterrichtsformen im Englischunterri...,Weingarten,Weingarten (Landkreis Ravensburg),4065156-3,both
68,1000904997,Sedimentationsverhalten von Submikrometerparti...,Talca/Chile,2187.0,1000904997,Sedimentationsverhalten von Submikrometerparti...,Talca/Chile,Talca,4806806-8,both
85,100502474X,Efficient index structures for and application...,Saarbrückenk,6442.0,100502474X,Efficient index structures for and application...,Saarbrückenk,Saarbrücken,4076912-4,both
89,1006817387,Tone-noise dichotomy : investigating tonal con...,Oldenburg,7901.0,1006817387,Tone-noise dichotomy : investigating tonal con...,Oldenburg,Oldenburg (Oldenburg),4043457-6,both
114,1031199470,Erfassung arbeitsbedingter Kniebelastungen an ...,St. Augustin,38664.0,1031199470,Erfassung arbeitsbedingter Kniebelastungen an ...,St. Augustin,Sankt Augustin,4105316-3,both
149,1187978159,Geistig behinderte Menschen in der DDR : Erzie...,Oberhausen,140357.0,1187978159,Geistig behinderte Menschen in der DDR : Erzie...,Oberhausen,Oberhausen (Rheinland),4042927-1,both
187,962138738,Struktur und Entstehung von Komplexen aus kati...,S.l.] @,231791.0,962138738,Struktur und Entstehung von Komplexen aus kati...,S.l.] @,unknown,NaN,both
194,964930307,"Subsistenzformen, Siedlungsweisen und Prozesse...",12,237577.0,964930307,"Subsistenzformen, Siedlungsweisen und Prozesse...",12,unknown,NaN,both
206,970331886,Der Brantas,11,249061.0,970331886,Der Brantas,11,unknown,NaN,both
215,971944253,X-ray and radio variability of Cygnus X-1,Rottenburg,252473.0,971944253,X-ray and radio variability of Cygnus X-1,Rottenburg,Rottenburg am Neckar,4050733-6,both


In [16]:
places = gnd_places['GND-ID'].to_list()
print("Anzahl GND-IDs in Places: ", len(places))

Anzahl GND-IDs in Places:  270


In [18]:
## Jetzt noch Geokoordinaten über SRU der GND abfragen und hinzufügen: 

def gnb_sru(gndid):
    
    base_url = "https://services.dnb.de/sru/authorities"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': 'nid="' + str(gndid) + '"'
         }
    
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Authority'})
    #print("Treffer: ", len(records))
    treffer = xml.find_all("numberOfRecords")[0].text
    #print("number of records: ", treffer)
    
    if len(records) == 1: 
        
        #print("Es geht weiter")
        for record in records:
            content = record
    
        ns = {"marc":"http://www.loc.gov/MARC21/slim"}
        xml1 = ET.fromstring(unicodedata.normalize("NFC", str(content)))
        long = xml1.xpath("marc:datafield[@tag = '034']/marc:subfield[@code = 'd']", namespaces=ns)
        lat = xml1.xpath("marc:datafield[@tag = '034']/marc:subfield[@code = 'f']", namespaces=ns)
        for elem in long:
            if "." in elem.text: 
                long = elem.text
                if long.startswith("E"):
                    long = long.replace("E", "")
                    long = long.lstrip('0') 
                elif long.startswith("W"):
                    long = long.replace("W", "")
                    long = long.lstrip('0') 
                    long = "-" + long
                #print(long)
        for elem in lat:
            if "." in elem.text: 
                lat = elem.text
                if lat.startswith("N"):
                    lat = lat.replace("N", "")
                    lat = lat.lstrip('0') 
                elif lat.startswith("S"):
                    lat = lat.replace("S", "")
                    lat = lat.lstrip('0') 
                    lat = "-" + lat
                #print(lat)
        
        coordinates = {'GND-ID': gndid, 'lat': lat, 'long': long}
        return coordinates
             
    else:
        
        print("ERROR - ", gndid)
        coordinates = {'GND-ID': gndid, 'lat': ' ', 'long': ' '}
        return coordinates
        

In [19]:
myquery = [gnb_sru(place) for place in places]


ERROR -  nan
ERROR -  nan
ERROR -  nan
ERROR -  nan
ERROR -  nan


In [20]:
df2 = pd.DataFrame(myquery)
df2

,GND-ID,lat,long
0,4018272-1,47.995833,7.852222
1,4013255-9,51.221719,6.776160
2,4005056-7,49.947777,11.578888
3,4013284-5,51.432222,6.765000
4,4021965-3,54.093110,13.387859
...,...,...,...
265,4092713-1,48.069189,11.377029
266,4014830-0,52.218330,6.895830
267,4040188-1,47.333449,-79.432969
268,4278974-6,50.511666,12.400833


In [21]:
new = pd.merge(gnd_places, df2, on="GND-ID", how="left")
combined = new[["Place", "Abgleich", "GND", "GND-ID", "lat", "long"]].copy()
combined = combined.drop_duplicates("Place")
combined

,Place,Abgleich,GND,GND-ID,lat,long
0,Freiburg (Breisgau),Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222
4,Düsseldorf,Düsseldorf,Düsseldorf,4013255-9,51.221719,6.776160
9,Bayreuth,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888
10,Duisburg,Duisburg,Duisburg,4013284-5,51.432222,6.765000
14,Greifswald,Greifswald,Greifswald,4021965-3,54.093110,13.387859
...,...,...,...,...,...,...
677,Gauting,Gauting,Gauting,4092713-1,48.069189,11.377029
678,Enschede,Enschede,Enschede,4014830-0,52.218330,6.895830
679,Montreal,Montréal,Montréal,4040188-1,47.333449,-79.432969
680,Auerbach/Voigtland,Auerbach/Vogtl.,Auerbach/Vogtl.,4278974-6,50.511666,12.400833


In [22]:
combined.to_excel("data/geoplaces_coordinates_from_gnd.xlsx")

In [23]:
complete = pd.merge(data, combined, on="Place", how ="left")

In [24]:
complete["count"] = complete.groupby('GND-ID')['GND-ID'].transform('count')
complete

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008,Abgleich,GND,GND-ID,lat,long,count
0,1000000389,"Klöfkorn, Robert",Numerics for evolution equations : a general i...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,510,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng,Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222,7172.0
1,1000006395,"Speckmann, Bodo",Regulation der hepatischen Biosynthese von Sel...,ger,Düsseldorf,2009,570,570,DE-101,573.3845554,540,"Düsseldorf, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger,Düsseldorf,Düsseldorf,4013255-9,51.221719,6.776160,7521.0
2,100000645X,"Sauheitl, Leopold",Direct amino acid uptake by plants related to ...,eng,Bayreuth,2009,N/A,N/A,N/A,570,N/A,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
3,1000006948,"Sanchez, Martin",Charakterisierung des Replikationsenzyms ORF90...,ger,Bayreuth,2010,570,570,DE-101,572.86453629321,N/A,"Bayreuth, Univ., Diss., 2010",N/A,100204s2009 gw |||||om||| 00||||ger,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
4,1000007138,"Reimann, Bodo",Numerische Untersuchung der Stoßrohrströmung ...,ger,Duisburg,2009,620,620,DE-101,629.4152,620,"Duisburg, Essen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger,Duisburg,Duisburg,4013284-5,51.432222,6.765000,4844.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324777,999997297,"Moschkau, Peter",Defektabhängige Transporteigenschaften von Pr...,ger,Göttingen,2009,N/A,N/A,N/A,530,N/A,"Göttingen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger,Göttingen,Göttingen,4021477-1,51.534166,9.932222,10529.0
324778,999997505,"Scheler, Esther",Tailoring fluorene-based oligomers for micron ...,eng,Bayreuth,2009,540,540,DE-101,547.70455,VN 5927,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
324779,99999767X,"Körzdörfer, Thomas",Self-interaction and charge transfer in organi...,eng,Bayreuth,2009,530,530,DE-101,537.6223,UM 1200,"Bayreuth, Univ., Diss., 2009",Enth. 4 Sonderabdr. aus verschiedenen Zeitschr.,100204s2009 gw |||||om||| 00||||eng,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
324780,999999389,"Xystrakis, Fotios",The drought tolerance limit of European beec...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,630,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",Dateien in unterschiedlichen Formaten,100204s2009 gw |||||om||| 00||||eng,Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222,7172.0


In [25]:
places = complete[["Place", "Abgleich", "GND", "GND-ID", "lat", "long", "count"]]
places = places.drop_duplicates("GND-ID")
places = places[places.lat != "[]"]
places = places[places.lat != ""]

In [26]:
places['lat'] = places['lat'].str.replace(r'^[0]*', '', regex=True)
places['long'] = places['long'].str.replace(r'^[0]*', '', regex=True)
#places['count'] = places['count'].str.replace(".0", "")
places

,Place,Abgleich,GND,GND-ID,lat,long,count
0,Freiburg (Breisgau),Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222,7172.0
1,Düsseldorf,Düsseldorf,Düsseldorf,4013255-9,51.221719,6.776160,7521.0
2,Bayreuth,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
4,Duisburg,Duisburg,Duisburg,4013284-5,51.432222,6.765000,4844.0
5,Greifswald,Greifswald,Greifswald,4021965-3,54.093110,13.387859,3042.0
...,...,...,...,...,...,...,...
319158,Stade,Stade,Stade,4056709-6,53.593370,9.476290,1.0
319631,Luxembourg,Luxemburg,Luxemburg,4036728-9,49.750000,6.166669,1.0
320654,Göppingen,Göppingen,Göppingen,4021434-5,48.702800,9.654880,1.0
324293,Oberpframmern,Oberpframmern,Oberpframmern,4363260-9,48.022019,11.813309,1.0


In [27]:
places = places[places.lat != ""]
places = places[(places['lat'].notnull())]
places

,Place,Abgleich,GND,GND-ID,lat,long,count
0,Freiburg (Breisgau),Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222,7172.0
1,Düsseldorf,Düsseldorf,Düsseldorf,4013255-9,51.221719,6.776160,7521.0
2,Bayreuth,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1868.0
4,Duisburg,Duisburg,Duisburg,4013284-5,51.432222,6.765000,4844.0
5,Greifswald,Greifswald,Greifswald,4021965-3,54.093110,13.387859,3042.0
...,...,...,...,...,...,...,...
319158,Stade,Stade,Stade,4056709-6,53.593370,9.476290,1.0
319631,Luxembourg,Luxemburg,Luxemburg,4036728-9,49.750000,6.166669,1.0
320654,Göppingen,Göppingen,Göppingen,4021434-5,48.702800,9.654880,1.0
324293,Oberpframmern,Oberpframmern,Oberpframmern,4363260-9,48.022019,11.813309,1.0


In [28]:
places.to_excel("data/geoplaces.xlsx")

In [29]:
places.to_json("data/geoplaces.json")

## OLD

In [10]:
## Connect to Geolocator:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")

In [11]:
#Add new colum 'gcode' that contains translated Ort-value to geolocation: 
#WARNING: THIS MAY TAKE A WHILE

dfplaces['gcode'] = dfplaces.Place.apply(geolocator.geocode)
dfplaces

,Place,gcode
190,Freiburg (Breisgau),"(Freiburg (Breisgau) Hauptbahnhof, 5-7, Bismar..."
191,Düsseldorf,"(Düsseldorf, Nordrhein-Westfalen, Deutschland,..."
192,Bayreuth,"(Bayreuth, Bayern, Deutschland, (49.9427202, 1..."
194,Duisburg,"(Duisburg, Nordrhein-Westfalen, Deutschland, (..."
195,Greifswald,"(Greifswald, Vorpommern-Greifswald, Mecklenbur..."
...,...,...
305972,Stade,"(Stade, Niedersachsen, Deutschland, (53.599794..."
306446,Luxembourg,"(Luxembourg, Canton Luxembourg, Lëtzebuerg, (4..."
307471,Göppingen,"(Göppingen, Vereinbarte Verwaltungsgemeinschaf..."
311118,Oberpframmern,"(Oberpframmern, Glonn (VGem), Landkreis Ebersb..."


In [12]:
## Find entries where gcode didn't find a match: 

dfmissmatch = dfplaces[dfplaces['gcode'].isnull()]
dfmissmatch

,Place,gcode


In [13]:
#Split into long and lat: 
dfplaces['lat'] = [g.latitude for g in dfplaces.gcode]
dfplaces['lon'] = [g.longitude for g in dfplaces.gcode]

In [14]:
dfmerge = pd.merge(df, dfplaces, on='Place', how='left')
dfmerge["count"] = dfmerge.groupby('Place')['Place'].transform('count')
#dfmerge.to_csv("check_places.csv")
dfnew = dfmerge.drop_duplicates(subset=['Place'])
dfnew

,ID,Title,Place,gcode,lat,lon,count
0,1000000389,Numerics for evolution equations : a general i...,Freiburg (Breisgau),"(Freiburg (Breisgau) Hauptbahnhof, 5-7, Bismar...",47.997731,7.841295,6948
1,1000006395,Regulation der hepatischen Biosynthese von Sel...,Düsseldorf,"(Düsseldorf, Nordrhein-Westfalen, Deutschland,...",51.225402,6.776314,7157
2,100000645X,Direct amino acid uptake by plants related to ...,Bayreuth,"(Bayreuth, Bayern, Deutschland, (49.9427202, 1...",49.942720,11.576308,1749
4,1000007138,Numerische Untersuchung der Stoßrohrströmung ...,Duisburg,"(Duisburg, Nordrhein-Westfalen, Deutschland, (...",51.434999,6.759562,4578
5,1000007820,Synthese funktionalisierter Trinukleotide und ...,Greifswald,"(Greifswald, Vorpommern-Greifswald, Mecklenbur...",54.095791,13.381524,3042
...,...,...,...,...,...,...,...
305414,995983364,Leben und Werk des Arztes Christian Ernst Endt...,Stade,"(Stade, Niedersachsen, Deutschland, (53.599794...",53.599794,9.475438,1
305887,996223533,Crosswalking EUR-Lex : a proposal for a metada...,Luxembourg,"(Luxembourg, Canton Luxembourg, Lëtzebuerg, (4...",49.611277,6.129799,1
306910,997079479,"Liebhaberin, Königin, Zauberfrau : Studien zu...",Göppingen,"(Göppingen, Vereinbarte Verwaltungsgemeinschaf...",48.703138,9.654112,1
310549,999726307,Eine systemorientierte Managementmethode fü...,Oberpframmern,"(Oberpframmern, Glonn (VGem), Landkreis Ebersb...",48.022027,11.813629,1


In [15]:
dfnew.to_json("data/geoplaces.json")

In [20]:
test = pd.read_json("data/geoplaces.json")

In [21]:
test

,Place,Abgleich,GND,GND-ID,lat,long,count
0,Freiburg (Breisgau),Freiburg im Breisgau,Freiburg im Breisgau,4018272-1,47.995833,7.852222,7037.0
1,Düsseldorf,Düsseldorf,Düsseldorf,4013255-9,51.221719,6.776160,7288.0
2,Bayreuth,Bayreuth,Bayreuth,4005056-7,49.947777,11.578888,1790.0
4,Duisburg,Duisburg,Duisburg,4013284-5,51.432222,6.765000,4660.0
5,Greifswald,Greifswald,Greifswald,4021965-3,54.093110,13.387859,3042.0
...,...,...,...,...,...,...,...
310683,Stade,Stade,Stade,4056709-6,53.593370,9.476290,1.0
311156,Luxembourg,Luxemburg,Luxemburg,4036728-9,49.750000,6.166669,1.0
312179,Göppingen,Göppingen,Göppingen,4021434-5,48.702800,9.654880,1.0
315818,Oberpframmern,Oberpframmern,Oberpframmern,4363260-9,48.022019,11.813309,1.0
